In [2]:
import pandas as pd
import nltk
import numpy as np
import keras
from keras import models
from keras import layers
from keras.models import Sequentia
import regex as re 
import string
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import StandardScaler

In [60]:
def get_X_matrix(X):
    tok = Tokenizer(num_words=1000)
    tok.fit_on_texts(X)
    sequences = tok.texts_to_sequences(X)
    return sequence.pad_sequences(sequences,maxlen=100)

In [79]:
def preprocessing_data(data, len_set):
    data = pd.read_csv(data,encoding='latin-1', header = None)
    lab0 = data[data[0] == 0][:len_set]
    data[0][data[0] == 4] = 1
    lab4 = data[data[0] == 1][:len_set]
    data = pd.concat([lab0,lab4])
    data = data.reset_index()
    data.pop('index')
    tweets = []
    for row in range(len(data)):
        tweets.append((data[5][row], data[0][row]))
    filtered_tweets = []
    X = []
    y = []
    porter=PorterStemmer()
    sc = StandardScaler()
    for (words, sentiment) in tweets:
        
        words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
        words_filtered = list(filter(lambda x: not x.startswith('http') and not x.startswith('@'), words_filtered))
        words_filtered = [re.sub(r"\p{P}+", "", word) for word in words_filtered ]
        stem_sentence=[]
        for word in words_filtered:
            stem_sentence.append(porter.stem(word))
        
        X.append(stem_sentence)
        y.append(sentiment)
    
    X = get_X_matrix(X)
    return sc.fit_transform(X),np.array(y)

In [ ]:
X_train, y_train = preprocessing_data('train_data.csv',100000)
X_test, y_test = preprocessing_data('test_data.csv', 1000)

In [81]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(100,)),
      tf.keras.layers.Embedding(1000, 32, input_length=100),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(50, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(50, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])
model.compile(optimizer=RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=100, shuffle = True, verbose=1)
model.evaluate(X_test, y_test, batch_size=100,verbose=1)

Epoch 1/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.6931 - accuracy: 0.5068
Epoch 2/10
2000/2000 [==============================] - 13s 7ms/step - loss: 0.6924 - accuracy: 0.5155
Epoch 3/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.6922 - accuracy: 0.5166
Epoch 4/10
2000/2000 [==============================] - 11s 6ms/step - loss: 0.6919 - accuracy: 0.5197
Epoch 5/10
2000/2000 [==============================] - 11s 6ms/step - loss: 0.6919 - accuracy: 0.5197
Epoch 6/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.6918 - accuracy: 0.5195
Epoch 7/10
2000/2000 [==============================] - 13s 6ms/step - loss: 0.6920 - accuracy: 0.5190
Epoch 8/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.6919 - accuracy: 0.5186
Epoch 9/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.6920 - accuracy: 0.5180
Epoch 10/10
4/4 [==============================] - 0s 3ms/step - loss: 0.

[0.665386974811554, 0.785515308380127]